In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 25.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [102 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [102 kB]
Fetched 204 kB in 1s (139 kB/s)  
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-5).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.


In [2]:
!conda install -y -c conda-forge py-xgboost

done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
WARNING conda.models.version:get_matcher(541): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.9.0.*, but conda is ignoring the .* and treating it as 1.9.0
- WARNING conda.models.version:get_matcher(541): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.6.0.*, but conda is ignoring the .* and treating it as 1.6.0
done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - py-xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_

In [5]:
# !conda install -y -c conda-forge py-xgboost=*=cuda*

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import faiss
from xgboost import XGBClassifier

In [8]:
df = pd.read_csv("../llamatales/story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [15]:
hidden_states_by_layer = {}
# NUM_PROMPTS = 10
NUM_PROMPTS = 2

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'../llamatales/hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
            # print(curr_hidden_states.shape)
            
            #By layer
            for layer in range(1):
                curr_layer_hidden_states = curr_hidden_states[layer][0].astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
                # print(curr_layer_hidden_states.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(curr_layer_hidden_states)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [curr_layer_hidden_states]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:26<00:00, 37.50it/s]


In [21]:
layer_hs_array = hidden_states_by_layer["layer_0"]
print(layer_hs_array[0].shape)
print(layer_hs_array[1].shape)

(270, 512)
(349, 512)


In [31]:
#Context-level analysis
min_story_len = min(df["len_new_story"]) + 50
for context_level in range(min_story_len):
    layer_hs_upto_context = np.array([np.array(layer_hs_array[i][:context_level]).flatten() for i in range(len(layer_hs_array))])
    print(layer_hs_upto_context.shape)
    layer_context_hs = layer_hs_upto_context.astype('float32')
    print(layer_context_hs.shape)

(2000, 0)
(2000, 0)
(2000, 512)
(2000, 512)
(2000, 1024)
(2000, 1024)
(2000, 1536)
(2000, 1536)
(2000, 2048)
(2000, 2048)
(2000, 2560)
(2000, 2560)
(2000, 3072)
(2000, 3072)
(2000, 3584)
(2000, 3584)
(2000, 4096)
(2000, 4096)
(2000, 4608)
(2000, 4608)
(2000, 5120)
(2000, 5120)
(2000, 5632)
(2000, 5632)
(2000, 6144)
(2000, 6144)
(2000, 6656)
(2000, 6656)
(2000, 7168)
(2000, 7168)
(2000, 7680)
(2000, 7680)
(2000, 8192)
(2000, 8192)
(2000, 8704)
(2000, 8704)
(2000, 9216)
(2000, 9216)
(2000, 9728)
(2000, 9728)
(2000, 10240)
(2000, 10240)
(2000, 10752)
(2000, 10752)
(2000, 11264)
(2000, 11264)
(2000, 11776)
(2000, 11776)
(2000, 12288)
(2000, 12288)
(2000, 12800)
(2000, 12800)
(2000, 13312)
(2000, 13312)
(2000, 13824)
(2000, 13824)
(2000, 14336)
(2000, 14336)
(2000, 14848)
(2000, 14848)
(2000, 15360)
(2000, 15360)
(2000, 15872)
(2000, 15872)
(2000, 16384)
(2000, 16384)
(2000, 16896)
(2000, 16896)
(2000, 17408)
(2000, 17408)
(2000, 17920)
(2000, 17920)
(2000,)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


ValueError: setting an array element with a sequence.

In [ ]:
classifier = self.MODEL_CLASSIFIER(**ht_best_params, seed = 42, objective = 'multi:softmax', eval_metric = "merror")
classifier.fit(self.X_train, self.train_y)
preds = classifier.predict(self.X_test)